In [3]:
import numpy as np
import random 

# defining Pauli matrices
Y = np.array([[1,-1j],[1j,1]])
Z = np.array([[1,0],[0,-1]])

# defining rotation gates
def Ry(A):
    M = np.array([[np.cos(A/2),-np.sin(A/2)],[np.sin(A/2), np.cos(A/2)]])
    return M
def Rz(A):
    M = np.array([[np.exp(-1j*A/2),0],[0,np.exp(1j*A/2)]])
    return M
    
# Parameterising the general unitary with rotation gates
def U (A0,A1,A2): # Arguments: Euler Angles
    M = Rz(A1)@Ry(A0)@Rz(A2)
    return M

# derivatives wrt A0,A1,A2
def DA0(A0,A1,A2):
    dA0 = (-1j/2)*(Rz(A1)@Y@Ry(A0)@Rz(A2))
    return dA0
def DA1(A0,A1,A2):
    dA1 = (-1j/2)*(Z@Rz(A1)@Ry(A0)@Rz(A2))
    return dA1
def DA2 (A0,A1,A2):
    dA2 = (-1j/2)*(Rz(A1)@Ry(A0)@Z@Rz(A2))
    return dA2

# generate random unitary 
def Urand():
    A0=random.uniform(0.0, 4*np.pi)
    A1=random.uniform(0.0, 4*np.pi)
    A2=random.uniform(0.0, 4*np.pi)
    Urand= U(A0,A1,A2)
    return Urand

# defining inverse
def inverse(M):
    M_inverse = M.transpose().conjugate()
    return M_inverse



In [4]:
# function to define RHS for each qubit
def Utbm(j, A):
    U1 = U(A[0,0],A[0,1],A[0,2])
    U2 = U(A[1,0],A[1,1],A[1,2])
    U3 = U(A[2,0],A[2,1],A[2,2])
    U4 = U(A[3,0],A[3,1],A[3,2])
    U5 = U(A[4,0],A[4,1],A[4,2])
    U6 = U(A[5,0],A[5,1],A[5,2])
    U7 = U(A[6,0],A[6,1],A[6,2])
    U8 = U(A[7,0],A[7,1],A[7,2])
    Ulist = [U1,U2,U3,U4,U5,U6,U7,U8]
    Ulist[j-1] = np.eye(2)
    U12 = np.kron(Ulist[0],Ulist[1])
    U34 = np.kron(Ulist[2],Ulist[3])
    U56 = np.kron(Ulist[4],Ulist[5])
    U78 = np.kron(Ulist[6],Ulist[7])
    if j==1:
        Uf = np.eye(4)
        Ub = U12@CNOT@U34@CNOT@U56@CNOT@U78
    elif j==2:
        Uf = U12
        Ub = CNOT@U34@CNOT@U56@CNOT@U78
    elif j==3:
        Uf = U12@CNOT
        Ub =U34@CNOT@U56@CNOT@U78
    elif j==4:
        Uf = U12@CNOT@U34
        Ub =CNOT@U56@CNOT@U78
    elif j==5:
        Uf = U12@CNOT@U34@CNOT
        Ub = U56@CNOT@U78
    elif j==6:
        Uf = U12@CNOT@U34@CNOT@U56
        Ub = CNOT@U78
    elif j==7:
        Uf = U12@CNOT@U34@CNOT@U56@CNOT
        Ub = U78
    elif j==8:
        Uf = U12@CNOT@U34@CNOT@U56@CNOT@U78
        Ub = np.eye(4)
    U_tbm = inverse(Uf)@U_goal@inverse(Ub)
    return U_tbm

In [16]:
def gradient_descent(A, iterations,alpha):
    for j in range(1,9):
        Ui = Utbm(j,A)
        A0=random.uniform(0.0, 4*np.pi)
        A1=random.uniform(0.0, 4*np.pi)
        A2=random.uniform(0.0, 4*np.pi)
        #print("initial:",A0,A1,A2)
        U_initial= U(A0,A1,A2)
        if j%2!=0:
            Uxyz = np.kron(U_initial,np.eye(2))
            #print(Uxyz)
            #print(j)
        else:
            Uxyz = np.kron(np.eye(2), U_initial)
            #print(Uxyz)
            #print(j)
        for i in range(iterations):
            Tr = np.trace(Ui@inverse(Uxyz))
            Abs = np.abs(Tr)
            Uerr = 4 - Abs
            #print(Abs/Tr)
            #print(Uerr)
            if Uerr <= 1e-8:
                break
            else:
                dA0 = DA0(A0,A1,A2)
                dA1 = DA1(A0,A1,A2)
                dA2 = DA2(A0,A1,A2)
                if j%2!=0:
                    UdA0 = np.kron(dA0,np.eye(2))
                    UdA1 = np.kron(dA1,np.eye(2))
                    UdA2 = np.kron(dA2,np.eye(2))
                else:
                    UdA0 = np.kron(np.eye(2),dA0)
                    UdA1 = np.kron(np.eye(2),dA1)
                    UdA2 = np.kron(np.eye(2),dA2)
                Grad_A0 = -(Abs/Tr)*(np.trace(Ui@inverse(UdA0)))
                Grad_A1 = -(Abs/Tr)*(np.trace(Ui@inverse(UdA1)))
                Grad_A2 = -(Abs/Tr)*(np.trace(Ui@inverse(UdA2)))
                A0 = A0 - alpha*Grad_A0
                A1 = A1 - alpha*Grad_A1
                A2 = A2 - alpha*Grad_A2
                if j%2 != 0:
                    Uxyz = np.kron(U(A0,A1,A2),np.eye(2))
                else:
                    Uxyz = np.kron(np.eye(2), U(A0,A1,A2) )
            #print(Uxyz)
            A[j-1][0]=A0
            A[j-1][1]=A1
            A[j-1][2]=A2
    return A

In [17]:
CNOT = [[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]] # switch tensor control

# goal unitary
UA= Urand()
UB= Urand()
U_goal = np.kron(UA,UB) 

A = (np.zeros((8,3),dtype = complex))
for i in range(8):
    for j in range(3):
        A[i][j] = random.uniform(0.0, 4*np.pi) 

AOpt = gradient_descent(A,1000000,0.01)

#A_result = gradient_descent(U_tbm,1000,0.01)

#verification
U_1 = U(AOpt[0][0],AOpt[0][1],AOpt[0][2])
U_2 = U(AOpt[1][0],AOpt[1][1],AOpt[1][2])
U_3 = U(AOpt[2][0],AOpt[2][1],AOpt[2][2])
U_4 = U(AOpt[3][0],AOpt[3][1],AOpt[3][2])
U_5 = U(AOpt[4][0],AOpt[4][1],AOpt[4][2])
U_6 = U(AOpt[5][0],AOpt[5][1],AOpt[5][2])
U_7 = U(AOpt[6][0],AOpt[6][1],AOpt[6][2])
U_8 = U(AOpt[7][0],AOpt[7][1],AOpt[7][2])


U_12 = np.kron(U_1,U_2)
U_34 = np.kron(U_3,U_4)
U_56 = np.kron(U_5,U_6)
U_78 = np.kron(U_7,U_8)
U_found = U_12@CNOT@U_34@CNOT@U_56@CNOT@U_78
print(U_goal)
print(U_found)

[[-0.71805301+0.265414j    0.14932896-0.43779627j -0.37146205+0.09229569j
   0.09834947-0.2093213j ]
 [ 0.12147229+0.44632857j  0.69992022+0.31009095j  0.03575057+0.22849491j
   0.33071443+0.19269279j]
 [-0.33071443+0.19269279j  0.03575057-0.22849491j  0.69992022-0.31009095j
  -0.12147229+0.44632857j]
 [ 0.09834947+0.2093213j   0.37146205+0.09229569j -0.14932896-0.43779627j
  -0.71805301-0.265414j  ]]
[[ 0.24923582-0.34124543j  0.04093588-0.34708338j  0.3731521 +0.12461723j
   0.67760828+0.22565402j]
 [-0.06868229-0.17449634j -0.3098089 -0.23917466j  0.22782149-0.27881918j
   0.40704708-0.68080996j]
 [-0.00919958+1.28683115j  0.27364448+1.98968973j -3.6926894 +2.78585959j
  -2.45303478-1.65499021j]
 [ 0.05143052-0.09364947j  2.50537646+1.05621185j  0.19067185+3.27890389j
  -2.11477587+1.49294181j]]
